In [1]:
#load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize']=(10.0,8.0)
import seaborn as sns
from scipy import stats
from scipy.stats import norm

In [2]:
#load data
#import csv
train_text = pd.read_csv("training_text",sep='\|\|',engine='python',names=['Id','Text'])
train_var = pd.read_csv("training_variants")


In [3]:
test_text = pd.read_csv("test_text",sep='\|\|',engine='python',names=['Id','Text'],skiprows=1)
test_var = pd.read_csv("test_variants")
test=test_text.join(test_var)
test.drop('ID',axis=1,inplace=True)

test.head()

,Id,Text,Gene,Variation
0,0,2. This mutation resulted in a myeloproliferat...,ACSL4,R570S
1,1,Abstract The Large Tumor Suppressor 1 (LATS1)...,NAGLU,P521L
2,2,Vascular endothelial growth factor receptor (V...,PAH,L333F
3,3,Inflammatory myofibroblastic tumor (IMT) is a ...,ING1,A148D
4,4,Abstract Retinoblastoma is a pediatric retina...,TMEM216,G77A


In [4]:
train_var.head()

,ID,Gene,Variation,Class
0,0,FAM58A,Truncating Mutations,1
1,1,CBL,W802*,2
2,2,CBL,Q249E,2
3,3,CBL,N454D,3
4,4,CBL,L399V,4


In [5]:
train=train_text.join(train_var)
train.drop('ID',axis=1,inplace=True)
train.head()

,Id,Text,Gene,Variation,Class
0,0,Cyclin-dependent kinases (CDKs) regulate a var...,FAM58A,Truncating Mutations,1
1,1,Abstract Background Non-small cell lung canc...,CBL,W802*,2
2,2,Abstract Background Non-small cell lung canc...,CBL,Q249E,2
3,3,Recent evidence has demonstrated that acquired...,CBL,N454D,3
4,4,Oncogenic mutations in the monomeric Casitas B...,CBL,L399V,4


In [6]:
train.describe(include=[np.object])

,Text,Gene,Variation
count,3321,3321,3321
unique,1921,264,2996
top,The PTEN (phosphatase and tensin homolog) phos...,BRCA1,Truncating Mutations
freq,53,264,93


In [7]:
train.drop('Id',axis=1).describe()

,Class
count,3321.000000
mean,4.365854
std,2.309781
min,1.000000
25%,2.000000
50%,4.000000
75%,7.000000
max,9.000000


In [8]:
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems
from sklearn.feature_extraction import text 

stop = text.ENGLISH_STOP_WORDS.union(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}','%'])

In [ ]:
vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words=stop,lowercase=True)

In [ ]:
%%time
vec = vectorizer.fit_transform(train['Text'])

In [9]:
import pickle
#with open("train_vectorizer.pkl", 'wb') as handle:
#                    pickle.dump(vectorizer, handle)
#with open("train_transformed.pkl", 'wb') as handle:
#                    pickle.dump(vec, handle)

In [10]:
vectorizer = pickle.load(open("train_vectorizer.pkl", "rb" ) )
vec = pickle.load(open("train_transformed.pkl", "rb" ) )

In [11]:
vectorizerGene = TfidfVectorizer(tokenizer=tokenize, stop_words=stop,lowercase=True,vocabulary=vectorizer.get_feature_names())
vectorizerVariation = TfidfVectorizer(tokenizer=tokenize, stop_words=stop,lowercase=True,vocabulary=vectorizer.get_feature_names())

In [12]:
vecGene = vectorizerGene.fit_transform(train['Gene'])
vecVariation =  vectorizerVariation.fit_transform(train['Variation'])

In [13]:
import scipy.sparse as sps

In [14]:
new=sps.hstack((vec,vecGene,vecVariation))

In [ ]:
df_class= pd.get_dummies(train['Class'],prefix='Class')
df_class.head(10)

In [ ]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(200)
svd1 = svd.fit(new)
x_train = svd.transform(new)
#x_test_df = svd.transform(x_test_df1)
#sorted(svd.explained_variance_ratio_)

In [15]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
model = XGBClassifier(objective= 'multi:softprob', n_estimators=300, learning_rate=0.05)

C:\Users\manshubh.singh\AppData\Local\conda\conda\envs\my_root\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
%%time
xgbfit = model.fit(new,train['Class'],eval_metric='mlogloss')

Wall time: 11min 43s


In [33]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()
logistic.fit(new,train['Class'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
def stand(x):
    if x<0.01:
        return 0
    elif x>=0.8:
        return 1
    else:
        return x

In [25]:
len(vectorizer.get_feature_names())

164763

In [26]:
%%time
#testvectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words=stop,lowercase=True,vocabulary=vectorizer.get_feature_names())
testvec = vectorizer.transform(test['Text'])


Wall time: 47.6 s


In [ ]:
#with open("test_transformed.pkl", 'wb') as handle:
#    pickle.dump(testvec, handle)

In [ ]:
#testvec = pickle.load(open("test_transformed.pkl", "rb" ) )

In [27]:
#testvectorizerGene = TfidfVectorizer(tokenizer=tokenize, stop_words=stop,lowercase=True,vocabulary=vectorizer.get_feature_names())
#testvectorizerVariation = TfidfVectorizer(tokenizer=tokenize, stop_words=stop,lowercase=True,vocabulary=vectorizer.get_feature_names())

testvecGene = vectorizerGene.transform(test['Gene'])
testvecVariation =  vectorizerVariation.transform(test['Variation'])


In [28]:
testnew=sps.hstack((testvec,testvecGene,testvecVariation))

In [ ]:
testsvd = TruncatedSVD(200)
testsvd1 = svd.fit(testnew)
x_test = svd.transform(testnew)

In [29]:
xgbresult=pd.DataFrame()
xgbresult['ID']=test['Id']
c=['Class1','Class2','Class3','Class4','Class5','Class6','Class7','Class8','Class9',]
xgbresult=pd.DataFrame(index=test.Id,data=xgbfit.predict_proba(testnew),columns=['Class1','Class2','Class3','Class4','Class5','Class6','Class7','Class8','Class9',])
xgbresult[c]=xgbresult[c].applymap(stand)
xgbresult.head(10)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [34]:
result=pd.DataFrame()
result['ID']=test['Id']
c=['Class1','Class2','Class3','Class4','Class5','Class6','Class7','Class8','Class9',]
result=pd.DataFrame(index=test.Id,data=logistic.predict_proba(testnew),columns=['Class1','Class2','Class3','Class4','Class5','Class6','Class7','Class8','Class9',])
result[c]=result[c].applymap(stand)
result.head(10)

,Class1,Class2,Class3,Class4,Class5,Class6,Class7,Class8,Class9
Id,,,,,,,,,
0,0.076644,0.148582,0.012821,0.074536,0.026804,0.041971,0.607755,0.000000,0.000000
1,0.107029,0.105979,0.021699,0.400196,0.042282,0.070611,0.225187,0.011437,0.015581
2,0.120753,0.147269,0.023806,0.190128,0.048808,0.080391,0.368239,0.000000,0.011111
3,0.093343,0.138459,0.019343,0.111042,0.040817,0.036402,0.546123,0.000000,0.000000
4,0.139812,0.102265,0.015552,0.243389,0.042719,0.043820,0.397844,0.000000,0.000000
5,0.204074,0.156001,0.018578,0.341944,0.057200,0.054203,0.149570,0.000000,0.010566
6,0.141998,0.095250,0.038105,0.138310,0.040933,0.039379,0.492088,0.000000,0.000000
7,0.120423,0.137359,0.021970,0.328508,0.074515,0.090672,0.209947,0.000000,0.000000
8,0.267125,0.194583,0.019969,0.162667,0.039146,0.064035,0.230690,0.000000,0.012680


In [36]:
result.to_csv("Submission7.csv")

In [31]:
result.loc[5666]

NameError: name 'result' is not defined